In [3]:
cd drive/MyDrive/lecture/datasets

/content/drive/MyDrive/lecture/datasets


In [5]:
!git clone https://github.com/jungyeul/korean-parallel-corpora.git

Cloning into 'korean-parallel-corpora'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 168 (delta 18), reused 0 (delta 0), pack-reused 131
Receiving objects: 100% (168/168), 20.48 MiB | 10.28 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [6]:
ls

korean-parallel-corpora/


In [7]:
cd korean-parallel-corpora/

/content/drive/MyDrive/lecture/datasets/korean-parallel-corpora


In [8]:
ls

bible/    korean-english-jhe/      korean-french-jim-v1/         README.md
contrib/  korean-english-news-v1/  northkorean-english-news-v1/


In [12]:
ls

bible/    korean-english-jhe/      korean-french-jim-v1/         README.md
contrib/  korean-english-news-v1/  northkorean-english-news-v1/


In [13]:
! pip install sentencepiece 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 6.5 MB/s 


Library

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd

import sentencepiece as spm

import numpy as np

import time

from sklearn.metrics import mean_squared_error
import joblib

import os

from sklearn.model_selection import train_test_split

In [14]:
VOCAB_SIZE = 10000
SEQ_LEN = 60

In [15]:
!pwd

/content/drive/MyDrive/lecture/datasets/korean-parallel-corpora


In [22]:
DATASET_PATH = '/content/drive/MyDrive/lecture/datasets/korean-parallel-corpora/bible'

Data Load

In [23]:
#영어
en_train = open(os.path.join(DATASET_PATH,'bible-all.en.txt'))
en_train_content = en_train.read()
en_train_list = en_train_content.split('\n')

In [26]:
#한국어
ko_train = open(os.path.join(DATASET_PATH,'bible-all.kr.txt'))
ko_train_content = ko_train.read()
ko_train_list = ko_train_content.split('\n')

In [27]:
en_train_list[:10]

['Genesis1.1  In the beginning God created the heavens and the earth.',
 'Genesis1.2  Now the earth was formless and empty, darkness was over the surface of the deep, and the Spirit of God was hovering over the waters.',
 'Genesis1.3  And God said, "Let there be light," and there was light.',
 'Genesis1.4  God saw that the light was good, and He separated the light from the darkness.',
 'Genesis1.5  God called the light "day," and the darkness he called "night." And there was evening, and there was morning--the first day.',
 'Genesis1.6  And God said, "Let there be an expanse between the waters to separate water from water."',
 'Genesis1.7  So God made the expanse and separated the water under the expanse from the water above it. And it was so.',
 'Genesis1.8  God called the expanse "sky." And there was evening, and there was morning--the second day.',
 'Genesis1.9  And God said, "Let the water under the sky be gathered to one place, and let dry ground appear." And it was so.',
 'Genes

In [28]:
ko_train_list[:10]

['Genesis1.1  태초에 하나님이 천지를 창조하셨다.',
 'Genesis1.2  땅이 혼돈하고 공허하며, 어둠이 깊음 위에 있고, 하나님의 영은 물 위에 움직이고 계셨다.',
 'Genesis1.3  하나님이 말씀하시기를 "빛이 생겨라" 하시니, 빛이 생겼다.',
 'Genesis1.4  그 빛이 하나님 보시기에 좋았다. 하나님이 빛과 어둠을 나누셔서,',
 'Genesis1.5  빛을 낮이라고 하시고, 어둠을 밤이라고 하셨다. 저녁이 되고 아침이 되니, 하루가 지났다.',
 'Genesis1.6  하나님이 말씀하시기를 "물 한가운데 창공이 생겨, 물과 물 사이가 갈라져라" 하셨다.',
 'Genesis1.7  하나님이 이처럼 창공을 만드시고서, 물을 창공 아래에 있는 물과 창공 위에 있는 물로 나누시니, 그대로 되었다.',
 'Genesis1.8  하나님이 창공을 하늘이라고 하셨다. 저녁이 되고 아침이 되니, 이튿날이 지났다.',
 'Genesis1.9  하나님이 말씀하시기를 "하늘 아래에 있는 물은 한 곳으로 모이고, 뭍은 드러나거라" 하시니, 그대로 되었다.',
 'Genesis1.10  하나님이 뭍을 땅이라고 하시고, 모인 물을 바다라고 하셨다. 하나님 보시기에 좋았다.']

In [30]:
data = pd.DataFrame()
data['en_raw'] = en_train_list
data['ko_raw'] = ko_train_list

In [31]:
data.head()

,en_raw,ko_raw
0,Genesis1.1 In the beginning God created the h...,Genesis1.1 태초에 하나님이 천지를 창조하셨다.
1,Genesis1.2 Now the earth was formless and emp...,"Genesis1.2 땅이 혼돈하고 공허하며, 어둠이 깊음 위에 있고, 하나님의 영..."
2,"Genesis1.3 And God said, ""Let there be light,...","Genesis1.3 하나님이 말씀하시기를 ""빛이 생겨라"" 하시니, 빛이 생겼다."
3,"Genesis1.4 God saw that the light was good, a...","Genesis1.4 그 빛이 하나님 보시기에 좋았다. 하나님이 빛과 어둠을 나누셔서,"
4,"Genesis1.5 God called the light ""day,"" and th...","Genesis1.5 빛을 낮이라고 하시고, 어둠을 밤이라고 하셨다. 저녁이 되고 ..."


In [32]:
len(data)

31104

In [34]:
data = data.reset_index(drop = True)
data.head()

,en_raw,ko_raw
0,Genesis1.1 In the beginning God created the h...,Genesis1.1 태초에 하나님이 천지를 창조하셨다.
1,Genesis1.2 Now the earth was formless and emp...,"Genesis1.2 땅이 혼돈하고 공허하며, 어둠이 깊음 위에 있고, 하나님의 영..."
2,"Genesis1.3 And God said, ""Let there be light,...","Genesis1.3 하나님이 말씀하시기를 ""빛이 생겨라"" 하시니, 빛이 생겼다."
3,"Genesis1.4 God saw that the light was good, a...","Genesis1.4 그 빛이 하나님 보시기에 좋았다. 하나님이 빛과 어둠을 나누셔서,"
4,"Genesis1.5 God called the light ""day,"" and th...","Genesis1.5 빛을 낮이라고 하시고, 어둠을 밤이라고 하셨다. 저녁이 되고 ..."


In [35]:
data['en'] = data['en_raw'].apply(lambda x: x.split(' ')[1:])
data['en'] = data['en'].apply(lambda x: (' ').join(x))
data['ko'] = data['ko_raw'].apply(lambda x: x.split(' ')[1:])
data['ko'] = data['ko'].apply(lambda x: (' ').join(x))

In [36]:
data = data[['en','ko']]
data.head()

,en,ko
0,In the beginning God created the heavens and ...,태초에 하나님이 천지를 창조하셨다.
1,"Now the earth was formless and empty, darknes...","땅이 혼돈하고 공허하며, 어둠이 깊음 위에 있고, 하나님의 영은 물 위에 움직이고..."
2,"And God said, ""Let there be light,"" and there...","하나님이 말씀하시기를 ""빛이 생겨라"" 하시니, 빛이 생겼다."
3,"God saw that the light was good, and He separ...","그 빛이 하나님 보시기에 좋았다. 하나님이 빛과 어둠을 나누셔서,"
4,"God called the light ""day,"" and the darkness ...","빛을 낮이라고 하시고, 어둠을 밤이라고 하셨다. 저녁이 되고 아침이 되니, 하루가..."


In [37]:
with open('src.txt', mode = 'w', encoding='utf8') as f:
    f.write('\n'.join(data['en']))
with open('trg.txt', mode= 'w', encoding='utf8') as f:
    f.write('\n'.join(data['ko']))

Sentencepiece --> 문장 토큰화


In [38]:
corpus = "src.txt"
prefix = "src"
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={VOCAB_SIZE}" +
    " --model_type=bpe" +
    " --max_sentence_length=999999" +  
    " --pad_id=0 --pad_piece=[PAD]" +  
    " --unk_id=1 --unk_piece=[UNK]" +  
    " --bos_id=2 --bos_piece=[BOS]" +  
    " --eos_id=3 --eos_piece=[EOS]" +  
    " --user_defined_symbols=[SEP],[CLS],[MASK]");  

In [39]:
corpus = "trg.txt"
prefix = "trg"
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={VOCAB_SIZE}" +
    " --model_type=bpe" +
    " --max_sentence_length=999999" +  
    " --pad_id=0 --pad_piece=[PAD]" +  
    " --unk_id=1 --unk_piece=[UNK]" +  
    " --bos_id=2 --bos_piece=[BOS]" +  
    " --eos_id=3 --eos_piece=[EOS]" +  
    " --user_defined_symbols=[SEP],[CLS],[MASK]"); 

정수화

In [40]:
sp_src = spm.SentencePieceProcessor()
sp_src.Load('src.model')


for idx in range(3):
    sentence = data['en'][idx]
    print(sp_src.EncodeAsPieces(sentence))
    print(sp_src.EncodeAsIds(sentence))

['▁In', '▁the', '▁beginning', '▁God', '▁created', '▁the', '▁heavens', '▁and', '▁the', '▁earth', '.']
[502, 10, 2155, 133, 3212, 10, 1354, 19, 10, 458, 9961]
['▁Now', '▁the', '▁earth', '▁was', '▁form', 'less', '▁and', '▁empty', ',', '▁darkness', '▁was', '▁over', '▁the', '▁surface', '▁of', '▁the', '▁deep', ',', '▁and', '▁the', '▁Spirit', '▁of', '▁God', '▁was', '▁ho', 'vering', '▁over', '▁the', '▁waters', '.']
[589, 10, 458, 127, 3464, 636, 19, 3330, 9958, 1451, 127, 268, 10, 6810, 21, 10, 1685, 9958, 19, 10, 837, 21, 133, 127, 386, 8186, 268, 10, 1411, 9961]
['▁And', '▁God', '▁said', ',', '▁"', 'Let', '▁there', '▁be', '▁light', ',"', '▁and', '▁there', '▁was', '▁light', '.']
[288, 133, 150, 9958, 65, 1612, 250, 52, 897, 393, 19, 250, 127, 897, 9961]


In [41]:
def en_encode(tmpstr:str) -> np.array :
    tmpstr = np.array(sp_src.EncodeAsIds(tmpstr))
 
    if len(tmpstr) > SEQ_LEN :
        tmpstr = tmpstr[:SEQ_LEN]
   
    else :
        tmpstr = np.pad(tmpstr, (0, SEQ_LEN - len(tmpstr)), 'constant', constant_values = sp_src.pad_id())
    
    return tmpstr

In [42]:
src_data = data['en']

src_list = []

for idx in range(len(src_data)):
    src_list.append(en_encode(src_data[idx]))

src_list[0]

array([ 502,   10, 2155,  133, 3212,   10, 1354,   19,   10,  458, 9961,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

In [43]:
sp_trg = spm.SentencePieceProcessor()
sp_trg.Load('trg.model')


for idx in range(3):
    sentence = data['ko'][idx]
    print(sp_trg.EncodeAsPieces(sentence))
    print(sp_trg.EncodeAsIds(sentence))

['▁태', '초', '에', '▁하나님이', '▁천', '지를', '▁창조', '하셨다', '.']
[561, 9349, 8964, 213, 369, 513, 2208, 883, 8962]
['▁땅이', '▁혼', '돈', '하고', '▁공', '허', '하며', ',', '▁어둠이', '▁깊', '음', '▁위에', '▁있고', ',', '▁하나님의', '▁영', '은', '▁물', '▁위에', '▁움직', '이고', '▁계셨다', '.']
[1226, 1567, 9398, 106, 440, 9291, 455, 8961, 4716, 1114, 9043, 394, 696, 8961, 194, 153, 8978, 119, 394, 5214, 411, 4486, 8962]
['▁하나님이', '▁말씀하시기를', '▁"', '빛이', '▁생겨', '라', '"', '▁하시니', ',', '▁빛이', '▁생', '겼다', '.']
[213, 2045, 32, 7888, 5865, 8983, 8995, 2921, 8961, 3057, 171, 1450, 8962]


In [44]:
def ko_encode(tmpstr: str) -> np.array:
    tmpstr = np.array(sp_trg.EncodeAsIds(tmpstr))
    tmpstr = np.insert(tmpstr, 0, sp_trg.bos_id())

    if len(tmpstr) >= SEQ_LEN:
      tmpstr = tmpstr[:SEQ_LEN-1]
      tmpstr = np.pad(tmpstr, (0, 1),
                        'constant', constant_values=sp_trg.eos_id())
    else:
        tmpstr = np.pad(tmpstr, (0, 1),
                        'constant', constant_values=sp_trg.eos_id())
        tmpstr = np.pad(tmpstr, (0, SEQ_LEN - len(tmpstr)),
                        'constant', constant_values=sp_trg.pad_id())

    return tmpstr


In [45]:
trg_data = data['ko']

trg_list = []

for idx in range(len(trg_data)):
    trg_list.append(ko_encode(trg_data[idx]))   

trg_list[0]

array([   2,  561, 9349, 8964,  213,  369,  513, 2208,  883, 8962,    3,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

In [46]:
src_train, src_valid, trg_train, trg_valid = train_test_split(src_list, trg_list, test_size=0.2, random_state=42)

In [49]:
print(src_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
import joblib

joblib.dump(src_train, 'src_train.pkl')
joblib.dump(trg_train, 'trg_train.pkl')
joblib.dump(src_valid, 'src_valid.pkl')
joblib.dump(trg_valid, 'trg_valid.pkl')
joblib.dump(src_list, 'src_list.pkl')
joblib.dump(trg_list, 'trg_list.pkl')


['trg_list.pkl']

In [51]:
ls

bible-all.en.txt  src_list.pkl   src.txt        trg_list.pkl   trg.txt
bible-all.kr.txt  src.model      src_valid.pkl  trg.model      trg_valid.pkl
readme.md         src_train.pkl  src.vocab      trg_train.pkl  trg.vocab


In [52]:
ddir = './'
src_train_path = os.path.join(ddir,'src_train.pkl')
src_valid_path = os.path.join(ddir,'src_valid.pkl')
trg_train_path = os.path.join(ddir,'trg_train.pkl')
trg_valid_path = os.path.join(ddir,'trg_valid.pkl')

src_train = joblib.load(src_train_path)
src_valid = joblib.load(src_valid_path)
trg_train = joblib.load(trg_train_path)
trg_valid = joblib.load(trg_valid_path)

In [53]:
print(src_valid)

[array([ 908, 8928,   81,  293, 9958, 1015, 1289,   25,  475, 9958,  737,
         89,   81,  723,   19,  128,   10, 1416,  103,  166, 1191,   89,
         97,   25, 3902,   81,  293, 9961,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0]), array([ 297,   10,  164,  554,  167, 9958,   47,  127, 2383, 4839, 9976,
         47,  127, 3948,   25, 2357, 2371,   19,  338,  339, 6014,  726,
        287,   14,  238,   25, 1347,   97, 9961,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0]), array([ 432, 3023,   10, 1370,   19,  500,  545,   21,   10,  685, 9958,
         19,   89,   81, 4072,   47,  361,  101,   75,  128,   10, 2546,
         21,   10,  682,   25, 4381,   10,  6